# Liage entre dicotopo et le pouillé

In [1]:
#On importe nos dataframes nécessaires à la réinjection

import pandas as pd
import re
from unidecode import unidecode

#On importe les différents fichiers nécessaires. 

liage_po = pd.read_csv('../../utils/pouille/out/linking_out/liage_po7.csv', dtype=str)

In [2]:
# On ne garde que les départements utiles à notre liage

dpt_list = liage_po['dpt_code'].unique().tolist()
dpt_list.remove("none")
dpt_list.sort()
print(dpt_list)  

['01', '05', '07', '15', '25', '26', '30', '38', '39', '42', '43', '48', '51', '52', '54', '67', '68', '69', '70', '71', '73', '74', '88']


In [3]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import re

# On définit le folder path
main_folder = '../../data'

#les fichiers avec les ID dicotopo sont tous appelés output7.xml.
def is_valid_xml_filename(file_path):
    if os.path.basename(file_path) == 'output7.xml':
        try:
            tree = ET.parse(file_path)
            root = tree.getroot()
            dep_attribute = root.attrib.get('dep')
            if dep_attribute and dep_attribute.isdigit() and dep_attribute in dpt_list:
                return True
        except Exception as e:
            pass
    
    return False

# On crée une liste pour y mettre le data 
data = []

# on cherche nos output7.xml
for root_dir, _, files in os.walk(main_folder):
    for filename in files:
        file_path = os.path.join(root_dir, filename)
        if is_valid_xml_filename(file_path):
            try:
                xml_tree = ET.parse(file_path)
                root_element = xml_tree.getroot()
            except ET.ParseError as e:
                print(f"Error parsing XML file {file_path}: {e}")
                continue  # Au cas où il y ait une erreur; pour que nous puissions voir

            #on extrait le old-id (pour la provenance) et le id (pour l'identifiant)   
            for article in root_element.findall('.//article'):
                article_old_id = article.get('old-id')
                article_id = article.get('id')
                
                # on extrait notre vedette
                vedette = article.find('./vedette/sm').text if article.find('./vedette/sm') is not None else ''

                # on extrait tout de notre définition en trois colonnes; typologie, insee et localisation
                definition_typologie = ''
                definition_localisation = ''
                insee = ''
                definition_elements = article.findall('./definition/*')
                for element in definition_elements:
                    if element.tag == 'typologie':
                        definition_typologie = element.text
                    elif element.tag == 'localisation':
                        localisation_content = ET.tostring(element, encoding='unicode')
                        localisation_match = re.search(r'<localisation>(.*?)<\/localisation>', localisation_content)
                        if localisation_match:
                            definition_localisation = re.sub(r'<[^>]*>', '', localisation_match.group(1))
                        insee_elem = element.find('./commune[@insee]')
                        if insee_elem is not None:
                            insee = insee_elem.get('insee')

                data.append({
                    'id': article_id,
                    'old-id' : article_old_id,
                    'vedette': vedette,
                    'definition_typologie': definition_typologie,
                    'definition_localisation': definition_localisation,
                    'insee': insee
                })

# on crée un dataframe
columns = ['id', 'old-id', 'vedette', 'definition_typologie', 'definition_localisation', 'insee']
places = pd.DataFrame(data, columns=columns)

# on print
print(places)

               id      old-id               vedette definition_typologie  \
0       P73728002  DT42-00001          Abbaye (L'),               hameau   
1       P34308606  DT42-00002  Abbaye (Bois de l').                        
2       P79175330  DT42-00003            Abbenevis,               hameau   
3       P82547046  DT42-00004          Abbés (Les),               hameau   
4       P66101493  DT42-00005        Abeleys (Les),         lieu détruit   
...           ...         ...                   ...                  ...   
158425  P28370101  DT01-11264         Yon-Artemare,              commune   
158426  P36549981  DT01-11265                Ysard,             lieu dit   
158427  P46622413  DT01-11266        Yvrieux (Les),    localité disparue   
158428  P22817934  DT01-11267             Zcabuens,    localité détruite   
158429  P50218803  DT01-11268             Zintimel,                écart   

            definition_localisation  insee  
0                commune de Chazeau  42095

In [4]:
#On applique la fonction voulue par Olivier pour supprimer les accents etc

import re
from unidecode import unidecode

def replace_special_characters(text):
    text = unidecode(text)
    text = re.sub(r'[-\'(),.]', ' ', text)  # Remlace également les parenthèses et les virgules
    return text


liage_po['vedette'] = liage_po['vedette'].astype(str)
places['vedette'] = places['vedette'].astype(str)

liage_po['vedette'] = liage_po['vedette'].apply(replace_special_characters)
places['vedette'] = places['vedette'].apply(replace_special_characters)

In [5]:
file_path = '../../utils/pouille/resources/tokens_dicotopo.txt'

with open(file_path, 'r') as file:
    tokens = file.read() 

In [6]:
def filter_words(text, tokens):
    text = replace_special_characters(text)  # Appliquer la fonction replace_special_characters
    words = text.split()
    words = [word for word in words if not re.search(r'\b{}\b'.format(re.escape(word.lower())), tokens)]
    filtered_text = ' '.join(words).strip() 
    return filtered_text

liage_po['vedette'] = liage_po['vedette'].apply(lambda x: filter_words(x, tokens=tokens))
places['vedette'] = places['vedette'].apply(lambda x: filter_words(x, tokens=tokens))

In [7]:
places

,id,old-id,vedette,definition_typologie,definition_localisation,insee
0,P73728002,DT42-00001,Abbaye,hameau,commune de Chazeau,42095
1,P34308606,DT42-00002,Abbaye,,,
2,P79175330,DT42-00003,Abbenevis,hameau,commune de La Bénisson-Dieu,42016
3,P82547046,DT42-00004,Abbes,hameau,commune de Noailly,42157
4,P66101493,DT42-00005,Abeleys,lieu détruit,commune de Jeansagnère,42114
...,...,...,...,...,...,...
158425,P28370101,DT01-11264,Yon Artemare,commune,canton de Champagne,01079
158426,P36549981,DT01-11265,Ysard,lieu dit,commune de Pont-de-Vaux,01305
158427,P46622413,DT01-11266,Yvrieux,localité disparue,commune de Reyrieux,01322
158428,P22817934,DT01-11267,Zcabuens,localité détruite,près Miribel,01249


In [8]:
# dataframe à charger
liages_df = pd.DataFrame()

In [9]:
columns_to_drop_po = ['localisationde', 'dpt_code', 'canton_code', 'method', 'reference']

# Renommer la colonne 'vedette' en 'label'
places.rename(columns={'vedette': 'label'}, inplace=True)

In [10]:
# Extraction des articles de type commune (ceux qui n’ont pas de commune de localisation)

communes_df = liage_po[liage_po['localisationco'] == 'none']
communes_df = communes_df.drop(columns=columns_to_drop_po)

# Communes exact match

In [11]:
#on lie par exact match vedette / label

#on enlève les lignes sans insee_code de liage_po7: elles ne servent à rien
communes_df_nonan = communes_df.dropna(subset=['insee_code'])

liage_exact_communes = pd.merge(communes_df_nonan,
                      places,
                      how='inner',
                      left_on=['insee_code','vedette'],
                      right_on=['insee', 'label'])


# Check for duplicate entries
duplicates = liage_exact_communes.duplicated(['article_id'], keep=False)
if duplicates.any():
    liage_exact_communes = liage_exact_communes[~duplicates]
    
# Add a 'method' column with value 'dpt_exact'
liage_exact_communes['method_dicotopo'] = 'communes_exact'


# Update the linked_places_df dataframe
liages_df = pd.concat([liages_df, liage_exact_communes]).drop_duplicates()

In [12]:
liages_df

,article_id,vedette,localisationco,insee_code,id,old-id,label,definition_typologie,definition_localisation,insee,method_dicotopo
0,PO7-00550,Brens,none,01061,P62070231,DT01-07902,Brens,hameau,commune de Brens,01061,communes_exact
1,PO7-01519,Gex,none,01173,P49239214,DT01-10037,Gex,,commune de Gex,01173,communes_exact
2,PO7-02605,Peron,none,01288,P42575708,DT01-10000,Peron,hameau,commune de Peron,01288,communes_exact
3,PO7-03044,Champ,none,01341,P39638741,DT01-09268,Champ,"village, chef-lieu",commune de Saint-Champ-Chatonod,01341,communes_exact
4,PO7-03826,Thoiry,none,01419,P83855921,DT01-10056,Thoiry,grange,commune de Thoiry,01419,communes_exact
5,PO7-01820,Lacoux,none,01185,P43530514,DT01-05821,Lacoux,commune,canton d’Hauteville,01185,communes_exact
6,PO7-01946,Lompnes,none,01185,P97302780,DT01-06072,Lompnes,commune,canton d’Hauteville,01185,communes_exact
7,PO7-02484,Ochiaz,none,01091,P91474412,DT01-07586,Ochiaz,commune,canton de Châtillon-de-Michaille,01091,communes_exact
8,PO7-01950,Longecombe,none,01185,P82592767,DT01-06080,Longecombe,commune,canton d’Hauteville,01185,communes_exact
9,PO7-02579,Passin,none,01079,P66493169,DT01-07751,Passin,commune,canton de Champagne,01079,communes_exact


# Lieux dans des communes exact match

In [13]:
# Extraction des articles appartenant à une commune (ceux ont une commune de localisation)

localisationco_df = liage_po[liage_po['localisationco'] != 'none']
localisationco_df = localisationco_df.drop(columns=columns_to_drop_po)

In [14]:
#on lie par exact match vedette de la localisationco / label

#on enlève les lignes sans insee_code de liage_po7: elles ne servent à rien
localisationco_df_nonan = localisationco_df.dropna(subset=['insee_code'])

liage_exact_localisationco = pd.merge(localisationco_df_nonan,
                      places,
                      how='inner',
                      left_on=['insee_code','vedette'],
                      right_on=['insee', 'label'])


# Check for duplicate entries
duplicates = liage_exact_localisationco.duplicated(['article_id'], keep=False)
if duplicates.any():
    liage_exact_localisationco = liage_exact_localisationco[~duplicates]
    
# Add a 'method' column with value 'dpt_exact'
liage_exact_localisationco['method_dicotopo'] = 'localisationco_exact'


# Update the linked_places_df dataframe
liages_df = pd.concat([liages_df, liage_exact_localisationco]).drop_duplicates()

In [15]:
liages_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationco,insee_code,id,old-id,label,definition_typologie,definition_localisation,insee,method_dicotopo
nunique,206,184,141,167,206,206,184,70,168,167,2
count,206,206,206,206,206,206,206,206,206,206,206
size,206,206,206,206,206,206,206,206,206,206,206


In [16]:
liages_df

,article_id,vedette,localisationco,insee_code,id,old-id,label,definition_typologie,definition_localisation,insee,method_dicotopo
0,PO7-00550,Brens,none,01061,P62070231,DT01-07902,Brens,hameau,commune de Brens,01061,communes_exact
1,PO7-01519,Gex,none,01173,P49239214,DT01-10037,Gex,,commune de Gex,01173,communes_exact
2,PO7-02605,Peron,none,01288,P42575708,DT01-10000,Peron,hameau,commune de Peron,01288,communes_exact
3,PO7-03044,Champ,none,01341,P39638741,DT01-09268,Champ,"village, chef-lieu",commune de Saint-Champ-Chatonod,01341,communes_exact
4,PO7-03826,Thoiry,none,01419,P83855921,DT01-10056,Thoiry,grange,commune de Thoiry,01419,communes_exact
...,...,...,...,...,...,...,...,...,...,...,...
192,PO7-03065,Clement,Pradelles,43154,P84547797,DT43-07958,Clement,ferme,commune de Pradelles,43154,localisationco_exact
195,PO7-03174,Germain,Hauterives,26148,P95839997,DT26-10317,Germain,"village, paroisse et section",commune d’Hauterives,26148,localisationco_exact
198,PO7-03306,Mansuy,Toul,54528,P66880131,DT54-03346,Mansuy,faubourg,Toul,54528,localisationco_exact
199,PO7-01395,Ferte,Saint-Ambreuil,71384,P91386982,DT71-07772,Ferte,hameau,commune de Saint-Ambreuil,71384,localisationco_exact


# Fuzzy match communes

In [17]:
import pandas as pd
from thefuzz import fuzz

communes_df_nonan = communes_df_nonan[~communes_df_nonan.article_id.isin(liages_df['article_id'])]

liage_fuzzy_communes = pd.merge(communes_df_nonan,
                                places,
                                how='inner',
                                left_on=['insee_code'],
                                right_on=['insee'])

# Define a threshold for similarity (50%: tout ce qui est plus bas donne des résultats vraiment trop erronnés.)
similarity_threshold = 50

# Filter communes based on fuzzy matching with insee_code
def fuzzy_match(row):
    vedette = row['vedette']
    label = row['label']

    if isinstance(vedette, str) and not pd.isnull(vedette) and isinstance(label, str) and not pd.isnull(label):
        # Calculate the fuzzy similarity score
        similarity_score = fuzz.ratio(vedette, label)
        if similarity_score >= similarity_threshold:
            return True

    return False

liage_fuzzy_communes = liage_fuzzy_communes[liage_fuzzy_communes.apply(fuzzy_match, axis=1)]

# Add a 'method' column with value 'dicotopo_fuzzy'
liage_fuzzy_communes['method_dicotopo'] = 'communes_fuzzy'

# Update the linked_places_df dataframe
liages_df = pd.concat([liages_df, liage_fuzzy_communes]).drop_duplicates()

In [18]:
liage_fuzzy_communes.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationco,insee_code,id,old-id,label,definition_typologie,definition_localisation,insee,method_dicotopo
nunique,552,549,1,547,1380,1380,1255,166,706,547,1
count,1381,1381,1381,1381,1381,1381,1381,1381,1381,1381,1381
size,1381,1381,1381,1381,1381,1381,1381,1381,1381,1381,1381


In [19]:
liage_fuzzy_communes

,article_id,vedette,localisationco,insee_code,id,old-id,label,definition_typologie,definition_localisation,insee,method_dicotopo
2,PO7-00083,Ambleon,none,01006,P99827857,DT01-05807,Lac Ambleon,petit lac,commune d’Ambléon,01006,communes_fuzzy
54,PO7-00128,Arbignieu,none,01015,P20656860,DT01-09742,Sellignieu,hameau,commune d’Arbignieu,01015,communes_fuzzy
55,PO7-00128,Arbignieu,none,01015,P54071260,DT01-09886,Sillignieu,hameau,commune d’Arbignieu,01015,communes_fuzzy
58,PO7-00128,Arbignieu,none,01015,P08621327,DT01-10759,Varigneux,lieu dit,commune d’Arbignieu,01015,communes_fuzzy
73,PO7-00146,Argis,none,01017,P39766075,DT01-07726,Paradis,granges,commune d’Argis,01017,communes_fuzzy
...,...,...,...,...,...,...,...,...,...,...,...
35221,PO7-03832,Thons,none,88471,P20574841,DT88-07912,Thons,hameau,commune des Thons,88471,communes_fuzzy
35224,PO7-03832,Thons,none,88471,P49077687,DT88-11265,Moulin Thons,papeterie,commune des Thons,88471,communes_fuzzy
35226,PO7-03832,Thons,none,88471,P25205178,DT88-12317,Thons,hameau,commune des Thons,88471,communes_fuzzy
35244,PO7-03892,Tremonzey,none,88479,P63194669,DT88-06018,Ferme Aymon,écart,commune de Trémonzey,88479,communes_fuzzy


# Fuzzy match localisationco

In [20]:

localisationco_df_nonan = localisationco_df_nonan[~localisationco_df_nonan.article_id.isin(liages_df['article_id'])]

liage_fuzzy_localisationco = pd.merge(localisationco_df_nonan,
                                places,
                                how='inner',
                                left_on=['insee_code'],
                                right_on=['insee'])


import pandas as pd
from thefuzz import fuzz

# Define a threshold for similarity (50%)
similarity_threshold = 50

# Filter localisationco_df based on fuzzy matching with insee_code
def fuzzy_match(row):
    vedette = row['vedette']
    label = row['label']

    if isinstance(vedette, str) and not pd.isnull(vedette) and isinstance(label, str) and not pd.isnull(label):
        # Calculate the fuzzy similarity score
        similarity_score = fuzz.ratio(vedette, label)
        if similarity_score >= similarity_threshold:
            return True

    return False

# Assuming you have defined liage_fuzzy_localisationco earlier
liage_fuzzy_localisationco = liage_fuzzy_localisationco[liage_fuzzy_localisationco.apply(fuzzy_match, axis=1)]

# Add a 'method' column with value 'dicotopo_fuzzy'
liage_fuzzy_localisationco['method_dicotopo'] = 'localisationco_fuzzy'

# Update the linked_places_df dataframe
liages_df = pd.concat([liages_df, liage_fuzzy_localisationco]).drop_duplicates()

In [21]:
liage_fuzzy_localisationco.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationco,insee_code,id,old-id,label,definition_typologie,definition_localisation,insee,method_dicotopo
nunique,104,96,92,92,344,344,314,92,131,92,1
count,346,346,346,346,346,346,346,346,346,346,346
size,346,346,346,346,346,346,346,346,346,346,346


# Exportation

In [22]:
# On crée un DataFrame new_rows_df en filtrant les lignes de liage_po qui ne sont pas présentes dans liages_df
new_rows_df = liage_po[~liage_po['article_id'].isin(liages_df['article_id'])]

# On vérifie si 'method_dicotopo' n'est pas dans les colonnes de new_rows_df
if 'method_dicotopo' not in new_rows_df.columns:
    new_rows_df['method_dicotopo'] = 'nulle'

# On utilise la méthode .concat pour ajouter les lignes de new_rows_df à liages_df
liages_df = pd.concat([liages_df, new_rows_df], ignore_index=True)


/var/folders/jj/dcr84y797818_4y88599z0500000gn/T/ipykernel_26755/772610514.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_rows_df['method_dicotopo'] = 'nulle'


In [23]:
liages_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationco,insee_code,id,old-id,label,definition_typologie,definition_localisation,insee,method_dicotopo,localisationde,dpt_code,canton_code,method,reference
nunique,3424,3109,532,2761,1894,1894,1669,234,874,665,5,22,20,320,5,2562
count,4496,4496,4496,4496,1933,1933,1933,1933,1933,1933,4496,2563,2563,2563,2563,2563
size,4496,4496,4496,4496,4496,4496,4496,4496,4496,4496,4496,4496,4496,4496,4496,4496


In [24]:
#on ajoute les lignes de notre pouillé qui n'ont pas eu de match avec dicotopo pour vérification manuelle

remaining_rows = liage_po[~liage_po['article_id'].isin(liages_df['article_id'])]
updated_liages_df = pd.concat([liages_df, remaining_rows], ignore_index=True)

#on sort par article_id pour plus de clarté
updated_liages_df = updated_liages_df.sort_values('article_id')
updated_liages_df = updated_liages_df.reset_index(drop=True)

In [25]:
updated_liages_df

,article_id,vedette,localisationco,insee_code,id,old-id,label,definition_typologie,definition_localisation,insee,method_dicotopo,localisationde,dpt_code,canton_code,method,reference
0,PO7-00001,Abbaye,Die,26113,P98573349,DT26-00002,Abbaye,ferme et ruines,commune de Die,26113,localisationco_exact,NaN,NaN,NaN,NaN,NaN
1,PO7-00002,Abbaye,Marnans,38221,NaN,NaN,NaN,NaN,NaN,NaN,nulle,Isère,38,Roybon,dpt_exact,Abbaye l Isère con de Roybon ...
2,PO7-00003,Abbaye,Saint-Jean-d’Aulph,74238,NaN,NaN,NaN,NaN,NaN,NaN,nulle,Haute-Savoie,74,Biot,dpt_fuzzy,Abbaye l Haute Savoie con du B...
3,PO7-00005,Abbaye Acey,Ougney,39398,NaN,NaN,NaN,NaN,NaN,NaN,nulle,Jura,39,Gendrey,dpt_exact,Abbaye d Acey l Jura con de Ge...
4,PO7-00006,Abbaye Grandvaux,Rivière-Devant,39258,NaN,NaN,NaN,NaN,NaN,NaN,nulle,Jura,39,Saint-Laurent,nulle,Abbaye de Grandvaux l Jura con...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4491,PO7-04282,Zellenberg,none,68383,P15742262,DT68-06243,Remelsperg,montagne,commune de Zellenberg,68383,communes_fuzzy,NaN,NaN,NaN,NaN,NaN
4492,PO7-04285,Zillisheim,none,68384,NaN,NaN,NaN,NaN,NaN,NaN,nulle,Haut-Rhin,68,Mulhouse,dpt_exact,Zillisheim Haut Rhin con de Mulho...
4493,PO7-04286,Zimmerbach,none,68385,P04264129,DT68-03113,Hilterspach,ruisseau,Zimmerbach,68385,communes_fuzzy,NaN,NaN,NaN,NaN,NaN
4494,PO7-04286,Zimmerbach,none,68385,P06067513,DT68-00549,Birckach,canton,territoire de Zimmerbach,68385,communes_fuzzy,NaN,NaN,NaN,NaN,NaN


In [26]:
updated_liages_df.agg(['nunique', 'count', 'size'])

,article_id,vedette,localisationco,insee_code,id,old-id,label,definition_typologie,definition_localisation,insee,method_dicotopo,localisationde,dpt_code,canton_code,method,reference
nunique,3424,3109,532,2761,1894,1894,1669,234,874,665,5,22,20,320,5,2562
count,4496,4496,4496,4496,1933,1933,1933,1933,1933,1933,4496,2563,2563,2563,2563,2563
size,4496,4496,4496,4496,4496,4496,4496,4496,4496,4496,4496,4496,4496,4496,4496,4496


In [27]:
# Sauvegarder le dataframe fusionné
updated_liages_df.to_csv('../../utils/pouille/out/reinjection_out/po7_dicotopo.csv', index=False)

In [28]:
#partie pour plus tard

# Réinsertion dans le fichier XML

In [29]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

# Charger le fichier XML
tree = ET.parse('../../utils/pouille/out/linking_out/PO_t7_modified.xml')
root = tree.getroot()
# Replace nan values with empty string in the DataFrame (juste pour le temps où olivier corrige)
liages_df = liages_df.replace({np.nan: ""})

In [30]:
# Pour chaque article dans le XML
for article in root.findall('article'):
    # Récupérer l'article_id
    article_id = article.get('old-id')

    # Vérifier si l'article_id est présent dans le DataFrame
    if article_id in liages_df['article_id'].values:
        # Récupérer la ligne correspondante dans le DataFrame
        liage_row = liages_df.loc[liages_df['article_id'] == article_id]

        # Récupérer la valeur de place_id
        dicotopo_code = liage_row['id'].values[0]

        # Ajouter l'attribut 'dicotopo' à la balise article
        article.set('dicotopo', dicotopo_code)

In [31]:
# Enregistrer le fichier XML modifié
tree.write('../../utils/pouille/out/reinjection_out/po7_dicotopo.xml', encoding='UTF-8', xml_declaration=True)